In [44]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

In [46]:
dfPitch = pd.read_csv("data/pitch_table.csv")
dfMaster = pd.read_csv("data/master.csv", encoding='latin-1')
dfStats = pd.read_csv("data/baseballReference2017Pitching.csv")

In [47]:
#merge datasets and clean
dfMerged = dfPitch.merge(dfMaster,how = "inner", left_on = "pit_id", right_on = "mlb_id")
threshold = 100 
value_counts = dfMerged["mlb_name"].value_counts() #
to_remove = value_counts[value_counts <= threshold].index
dfMerged["mlb_name"].replace(to_remove, np.nan, inplace=True)
dfMerged.dropna(subset = ["pitch_type" ,"mlb_name"],axis=0,inplace = True )
dfMerged = dfMerged.loc[dfMerged["type_conf"] >= 0.5]
dfMerged.reset_index(inplace=True)
dfWorking = dfMerged[["mlb_name","pitch_type","type_conf","zone","outs_ct","pa_ball_ct","pa_strike_ct","pa_event_cd","start_bases_cd","pa_terminal_fl","pitch_res"]]

In [48]:
dfStats = dfStats.merge(dfMaster,how = "inner", left_on = "playerID", right_on = "lahman_id")
dfStats = dfStats[["mlb_name","teamID","ERA","IP","SO","HR","WHIP"]]
dfStats.head()

,mlb_name,teamID,ERA,IP,SO,HR,WHIP
0,Jake Esch,SDP,0.00,0.0,0,0,0.000
1,Matt Koch,ARI,0.00,0.0,0,0,0.000
2,Pedro Beato,PHI,0.00,0.2,1,0,0.000
3,A.J. Minter,ATL,3.00,15.0,26,1,1.000
4,Ryan Madson,WSN,1.37,19.2,28,0,0.814


In [55]:
#coefficient ideas
#ball is bad
#X (in play) is bad 
#Called strike good
#Swinging strike best 
#foul is good but not much 
#homerun is bad 
#modifiers for count: matters more on 3 balls
#add up a score and divide by count 

#visualization 
#graph displaying all of the pitchers pitches and confidence rating
#dropdown menu to select player
#side by side/four pitchers to compare
#overall quality for all pitchers for a dropdown selected pitch 
#could also display ratio of strike/ball/in play in stacked graph 

#show pitchers normal stats underneath graphs - ERA, K, etc 

In [49]:
grouped = dfWorking.groupby(by = ["mlb_name","pitch_type"])

In [62]:
#calculate the pitch quality for each pitch type
#place more importance on PA terminal events, look at outs and base state 
#outs worth more with runners in scoring postion, hits worth more negative when there are runners on 
#
def pitch_quality(df): 
    res_dict = {'C': 0.3, 'S': 0.5, 'F': 0.1, 'B': -0.05, 'X': -0.1}
     
    modifier = 1
    pitch_result = df["pitch_res"]
    balls = df["pa_ball_ct"]
    event = df["pa_event_cd"]
    pa_term = df["pa_terminal_fl"]
    base_state = df["start_bases_cd"]
    qual = res_dict.get(pitch_result)
    
    #modifier increased if runners are in scoring position
    if base_state >= 2:
        modifier = 1.2
        
    #only consider the event cd if it is the final pitch of PA 
    #roughly based on 2* woba coefficients 
    #result of AB is more meaningful than individual pitches
    if pa_term == "T":
        if event == 23:
            #HR modifier
            qual -= 4.2
        if event == 20:
            #1B modifier
            qual -= 1.78
        if event == 21:
            #2B modifier
            qual -= 2.54
        if event == 22:
            #3B modifier
            qual -= 3.24
        if event == 14:
            #modifier for NIBB
            qual -= 1.8
        if event == 16:
            qual -= 0.8
            #modifier for HBP
        if event == 9:
            qual -= 0.2
            #modifier for WP
        if event == 2:
            #generic out modifier
            qual += 1.6
        if event == 3:
            #modifier for strikeout 
            qual += 2.2 
    
    
    return qual*modifier



dfWorking["weight"] = dfWorking.apply(pitch_quality,axis = 1)

In [58]:
#aggregate the percentages for ball, strike, foul, and inplay for each pitch type 

def ball_pct(df):
    count = df.loc[df == 'B'].count()
    return count/df.count()

def strike_pct(df):
    count = df.loc[(df == 'S') | (df == 'C')].count()
    return count/df.count()

def in_play_pct(df):
    count = df.loc[df == 'X'].count()
    return count/df.count()

def foul_pct(df): 
    count = df.loc[df == 'F'].count()
    return count/df.count()

dfWorking["quality"] = grouped["weight"].transform('mean')
dfWorking["pitch_count"] = grouped["weight"].transform('count')
dfWorking["ball_pct"] = grouped["pitch_res"].transform(ball_pct)
dfWorking["strike_pct"] = grouped["pitch_res"].transform(strike_pct)
dfWorking["in_play_pct"] = grouped["pitch_res"].transform(in_play_pct)
dfWorking["foul_pct"] = grouped["pitch_res"].transform(foul_pct)

In [59]:
#add average quality column for hovertool
dfWorking["avg_qual"] = dfWorking.groupby(by = "pitch_type")['quality'].transform('mean')

In [61]:
#write out the csv files for python script
pitch_names = {"FA": "Fastball","FF":"Four-seam Fastball", "FT": "Two-seam Fastball","FC": "Cutter", "FS":"Sinking Fastball","FO":"Pitch Out","SI":"Sinker","SL":"Slider","CU":"Curveball","KC":"Knuckle-curve","EP":"Eephus","CH":"Changeup","SC":"Screwball","KN":"Knuckleball"}
dfOut = dfWorking[["mlb_name","pitch_type","quality","ball_pct","strike_pct","foul_pct","in_play_pct","pitch_count","avg_qual"]]
dfOut = dfOut.drop_duplicates()
dfOut = dfOut.loc[dfOut["pitch_count"] > 10]
dfOut["pitch_name"] = dfOut["pitch_type"].replace(pitch_names)
dfOut.to_csv("data/pitch_quality.csv")
dfStats.to_csv("data/pitcher_stats.csv")
dfOut.head()

,mlb_name,pitch_type,quality,ball_pct,strike_pct,foul_pct,in_play_pct,pitch_count,avg_qual,pitch_name
0,Anthony Banda,FF,0.352091,0.331010,0.331010,0.146341,0.191638,287.0,0.272075,Four-seam Fastball
5,Anthony Banda,CH,0.199318,0.423529,0.258824,0.141176,0.176471,85.0,0.334053,Changeup
6,Anthony Banda,CU,0.241000,0.472973,0.256757,0.108108,0.162162,74.0,0.380957,Curveball
446,Jared Miller,FF,0.105056,0.527778,0.194444,0.111111,0.166667,36.0,0.272075,Four-seam Fastball
448,Jared Miller,SL,0.388750,0.500000,0.312500,0.000000,0.187500,16.0,0.411105,Slider


In [ ]:
#all other cells are were used to get Bokeh working for some of the plots before writing program

from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.layouts import row
from bokeh.palettes import Spectral11
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
src = ColumnDataSource(dfOut.groupby(by = ["mlb_name"]))

In [75]:

p = figure(x_range = 'mlb_name_pitch_type',y_range = (-1,1), plot_height=250, title="Fruit Counts", toolbar_location=None, tools="")
p.vbar(x = 'quality_mean',top = 'quality_mean', width=0.9,source = src)
#bar = figure(dfOut.loc[dfOut["mlb_name"] == "Zack Wheeler"], values='ball_pct', label='pitch_type', stack=['strike_pct'], agg='mean',
         # title="Python Interpreter Sampling", legend='top_right', plot_width=400)
output_notebook()
show(p)

Loading BokehJS ...

In [139]:
all_pitches = dfOut["pitch_type"].unique()

def make_dataset(name):
   # dfPlot = dfOut.loc[dfOut["mlb_name"] == name].drop_duplicates()
    name = np.array_str(dfPlot["mlb_name"].unique())[2:-2]
    pitches = dfPlot["pitch_type"].unique()
    source = ColumnDataSource(dfPlot)
    return source,pitches 

In [151]:
def make_plot(data, name, pitches,all_pitches): 
    
    p = figure(x_range=pitches, plot_height=250,plot_width = 400, toolbar_location=None, title= name, 
           y_axis_label = "Pitch Quality", x_axis_label = "Pitch Type")

    p.vbar(x='pitch_type', top='quality', width=0.9, source=data,line_color='white',
       fill_color=factor_cmap('pitch_type', palette=Spectral11, factors=all_pitches))
    return p
    

In [154]:
name = "Zack Wheeler"
name2 = "Zack Greinke"
data, pitches = make_dataset(name)
data2,pitches2 = make_dataset(name2)
p1 = make_plot(data,name,pitches,all_pitches)
p2 = make_plot(data2,name2,pitches2,all_pitches)

output_notebook()
show(row(p1,p2))

/anaconda/lib/python3.6/site-packages/bokeh/models/mappers.py:82: UserWarning: Palette length does not match number of factors. ['FO'] will be assigned to `nan_color` gray
  warnings.warn("Palette length does not match number of factors. %s will be assigned to `nan_color` %s" % (extra_factors, self.nan_color))


Loading BokehJS ...

In [198]:
from bokeh.io import output_file, show
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Select

output_notebook()
pitchers = dfOut["mlb_name"].drop_duplicates().tolist()
pitchers.sort(key=str.lower)
select = Select(title="Pitcher:", value="", options= pitchers)
select.on_change('value',function)

show(widgetbox(select))

Loading BokehJS ...

In [195]:
def function(attr,old,new):
    print(select.value)
    push_notebook()

In [194]:
def update(attr,old,new): 
    name = value
    new_src,pitches = make_dataset(name)
    data.data.update(new_src.data)
    push_notebook()

In [243]:
info = ["ball_pct","foul_pct","strike_pct","in_play_pct"]
headers = ["Ball %","Foul %","Strike %","In Play %"]
colors = ["#c9d9d3", "#718dbf", "#e84d60","#99d594"]

In [197]:
select.value

''

In [276]:
from bokeh.core.properties import value
#from bokeh.models import Legend
from bokeh.models import NumeralTickFormatter
output_notebook()
p = figure(x_range=pitches,y_range = (-.2,1), plot_height=250, title="Breakdown",
           toolbar_location=None, tools="")

p.vbar_stack(info, x='pitch_type', width=0.9, source=data,color = colors, legend = [value(x) for x in headers])
p.legend.location = 'bottom_right'
p.legend.orientation = 'horizontal'
p.legend.padding= 2
p.yaxis[0].formatter = NumeralTickFormatter(format="0.0%")
#legend=Legend(items = [value(x) for x in headers], location=(0, -30))
#p.add_layout(legend,'right')
show(p)

Loading BokehJS ...

In [ ]:
dfPlot.head()

In [13]:
dfStats = pd.read_csv("baseballReference2017Pitching.csv")
dfStats.head()

,playerID,yearID,teamID,lgID,W,L,ERA,G,GS,CG,...,HR,BB,IBB,SO,HBP,BK,WP,BFP,FIP,WHIP
0,eschja01,2017,SDP,NL,0,0,0.00,1,0,0,...,0,2,0,0,0,0,0,2,0.00,0.000
1,kochma01,2017,ARI,NL,0,0,0.00,1,0,0,...,0,1,0,0,0,0,0,3,0.00,0.000
2,beatope01,2017,PHI,NL,0,0,0.00,1,0,0,...,0,0,0,1,0,0,0,2,0.16,0.000
3,minteaj01,2017,ATL,NL,0,1,3.00,16,0,0,...,1,2,0,26,0,0,0,60,0.96,1.000
4,madsory01,2017,WSN,NL,3,0,1.37,20,0,0,...,0,3,0,28,2,0,0,75,1.07,0.814


In [26]:
dfOut.groupby(by = "pitch_type").mean()

,quality,ball_pct,strike_pct,foul_pct,in_play_pct,pitch_count,avg_qual
pitch_type,,,,,,,
CH,0.080802,0.434581,0.242057,0.138974,0.184388,135.845455,0.127654
CU,0.130296,0.441188,0.296246,0.117608,0.144957,153.207317,0.177786
EP,0.136114,0.502291,0.326580,0.059606,0.111522,34.166667,0.091415
FC,0.125775,0.392923,0.268141,0.158287,0.180649,183.769874,0.156538
FF,0.130778,0.367175,0.265518,0.193304,0.174002,406.188220,0.160365
FO,0.153333,0.404255,0.251773,0.177305,0.166667,282.000000,0.153333
FS,0.124663,0.427938,0.259403,0.140756,0.171903,228.100000,0.162321
FT,0.076960,0.376646,0.242687,0.178353,0.202314,222.268061,0.092966
KC,0.180849,0.416045,0.318086,0.132701,0.133168,336.380952,0.202818
